In [ ]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd

Use pandas to read the dataset.

In [40]:
! pwd
expr_df = pd.read_csv("../metadata/length_and_depth.csv")
expr_df = expr_df.drop("nvar", axis=1)
expr_df = expr_df.rename(columns={
    "length" : "expr_length",
    "depth" : "expr_depth"
})
expr_df

/Users/felixgao/Desktop/Year3/STA378/OptimizationParameterTuning/Julia Notebook/src


,problem,objtype,variable_nvar,expr_length,expr_depth
0,NZF1,least_squares,True,250,8
1,arglina,least_squares,True,10404,9
2,arglinb,least_squares,True,40200,8
3,arglinc,other,True,39204,7
4,argtrig,other,True,497,7
...,...,...,...,...,...
100,tridia,other,True,398,6
101,vardim,other,True,798,6
102,vibrbeam,least_squares,False,481,13
103,watson,least_squares,False,3900,12


In [71]:
complete_df = pd.read_csv("../results/complete_dataset_as_of_nov6.csv")
complete_df = complete_df[complete_df["is_init_run"] == 0]
complete_df = complete_df.rename(columns={"name" : "problem"})
df = complete_df.merge(
    expr_df,
    on = 'problem',
    how = "inner"
)
df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,neval_grad,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,315,0.000031,0.001440,5,False,False,least_squares,True,250,8
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,216,0.000019,0.001440,5,False,False,least_squares,True,250,8
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,228,0.000016,0.001440,5,False,False,least_squares,True,250,8
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,203,0.000017,0.001440,5,False,False,least_squares,True,250,8
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,225,0.000017,0.001440,5,False,False,least_squares,True,250,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,75,0.001356,0.008664,5,False,True,other,True,425,7
18822,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,75,0.001373,0.008664,5,False,True,other,True,425,7
18823,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,75,0.001367,0.008664,5,False,True,other,True,425,7
18824,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,75,0.001358,0.008664,5,False,True,other,True,425,7


In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# X = df_problem_level[[
#     "nvar", 
#     "expr_length", 
#     "expr_depth", 
#     "init_eval_obj_time", 
#     "init_eval_obj_mem",
#     "init_eval_obj_alloc",
#     "init_eval_grad_time",
#     "init_eval_grad_mem",
#     "init_eval_grad_alloc"
# ]].values
# y = df_problem_level["mem"].values     # categorical / integer

# clf = DecisionTreeClassifier(max_depth=5, random_state=0)
# clf.fit(X, y)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [ ]:
problems = df["problem"].unique()
rng = np.random.default_rng(seed=42)
rng.shuffle(problems)

n = len(problems)
n_train = int(0.7 * n)
n_valid = int(0.15 * n)

train_problems = problems[:n_train]
valid_problems = problems[n_train:n_train + n_valid]
test_problems  = problems[n_train + n_valid:]

train_df = df[df["problem"].isin(train_problems)].reset_index(drop=True)
valid_df = df[df["problem"].isin(valid_problems)].reset_index(drop=True)
test_df  = df[df["problem"].isin(test_problems)].reset_index(drop=True)

(105,)

In [122]:
from sklearn.tree import DecisionTreeRegressor
feature_cols = ["nvar", 
                "expr_length", 
                "expr_depth", 
                "mem",
                "init_eval_obj_time", 
                "init_eval_obj_mem",
                "init_eval_obj_alloc",
                "init_eval_grad_time",
                "init_eval_grad_mem",
                "init_eval_grad_alloc"]
target_col = "time"
X_train = train_df[feature_cols].to_numpy(dtype=float)
X_valid = valid_df[feature_cols].to_numpy(dtype=float)
X_test  = test_df[feature_cols].to_numpy(dtype=float)

t_train = train_df[target_col].to_numpy(dtype=float)
t_valid = valid_df[target_col].to_numpy(dtype=float)
t_test  = test_df[target_col].to_numpy(dtype=float)

In [138]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

best_model = None
best_score = np.inf
best_params = None

for max_depth in [3, 5, 7, 9, None]:
    for min_leaf in [1, 3, 5, 10]:
        model = DecisionTreeRegressor(
            max_depth=max_depth,
            min_samples_leaf=min_leaf,
            random_state=0,
        )
        model.fit(X_train, t_train)

        pred_valid = model.predict(X_valid)
        mse_valid = mean_squared_error(t_valid, pred_valid)

        if mse_valid < best_score:
            best_score = mse_valid
            best_model = model
            best_params = (max_depth, min_leaf)

print("best validation MSE:", best_score)
print("best params:", best_params)

y_pred_test = best_model.predict(X_test)

test_mse = mean_squared_error(t_test, y_pred_test)
test_mae = mean_absolute_error(t_test, y_pred_test)
test_r2  = r2_score(t_test, y_pred_test)

print("Test MSE:", test_mse)
print("Test MAE:", test_mae)
print("Test R^2:", test_r2)

best validation MSE: 2242.4577212734466
best params: (5, 5)
Test MSE: 4338.384093066318
Test MAE: 23.132366265660956
Test R^2: 0.09504450935978348


In [139]:
# after the loop
best_max_depth, best_min_leaf = best_params

X_train_full = np.vstack([X_train, X_valid])
t_train_full = np.concatenate([t_train, t_valid])

final_model = DecisionTreeRegressor(
    max_depth=best_max_depth,
    min_samples_leaf=best_min_leaf,
    random_state=0,
)
final_model.fit(X_train_full, t_train_full)

y_pred_test = final_model.predict(X_test)

test_mse = mean_squared_error(t_test, y_pred_test)
test_mae = mean_absolute_error(t_test, y_pred_test)
test_r2  = r2_score(t_test, y_pred_test)

print("Test MSE:", test_mse)
print("Test MAE:", test_mae)
print("Test R^2:", test_r2)


Test MSE: 4409.066295462132
Test MAE: 22.75686097079628
Test R^2: 0.08030071402574102


In [128]:
def choose_best_mem(model, x_problem, mem_candidates):
    preds = []
    for mem in mem_candidates:
        x = np.concatenate([x_problem, [mem]])
        preds.append((mem, model.predict(x.reshape(1, -1))[0]))
    return min(preds, key=lambda x: x[1])

In [129]:
problem_feature_cols = [c for c in feature_cols if c != "mem"]
mem_candidates = np.sort(train_df["mem"].unique())
for problem, group in valid_df.groupby("problem"):
    row0 = group.iloc[0]
    best_mem, pred_time = choose_best_mem(model, row0[problem_feature_cols].to_numpy(dtype=float), mem_candidates)
    print(f"problem {problem}: best_mem = {int(best_mem)}, predicted_time = {float(pred_time):.6f}")

problem bdqrtic: best_mem = 1, predicted_time = 1.366939
problem brybnd: best_mem = 1, predicted_time = 301.300821
problem clplatec: best_mem = 1, predicted_time = 1.270546
problem dixmaank: best_mem = 1, predicted_time = 1.270546
problem dixmaano: best_mem = 1, predicted_time = 1.270546
problem extrosnb: best_mem = 1, predicted_time = 3.590631
problem freuroth: best_mem = 1, predicted_time = 3.590631
problem genrose: best_mem = 1, predicted_time = 3.590631
problem integreq: best_mem = 1, predicted_time = 3.590631
problem ncb20b: best_mem = 1, predicted_time = 3.590631
problem palmer1c: best_mem = 1, predicted_time = 301.298061
problem powellsg: best_mem = 1, predicted_time = 1.411344
problem sinquad: best_mem = 1, predicted_time = 1.366939
problem sparsqur: best_mem = 1, predicted_time = 3.590631
problem watson: best_mem = 1, predicted_time = 301.298061


In [130]:
prob = valid_df["problem"].iloc[0]
group = valid_df[valid_df["problem"] == prob]

row0 = group.iloc[0]
x_problem = row0[problem_feature_cols].to_numpy(dtype=float)

for mem in [1, 2, 5, 10, 20, 50, 100]:
    x = np.concatenate([x_problem, [mem]])
    pred_time = model.predict(x.reshape(1, -1))[0]
    print(f"mem={mem}, predicted_time={pred_time}")


mem=1, predicted_time=1.3669388412
mem=2, predicted_time=1.3669388412
mem=5, predicted_time=1.3669388412
mem=10, predicted_time=1.3669388412
mem=20, predicted_time=1.3669388412
mem=50, predicted_time=1.3669388412
mem=100, predicted_time=1.3669388412
